In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from tensorflow import keras
from keras import layers
from keras import losses

print(tf.__version__)

2.14.0


## Построение модели
На вход идут данные по коррелирующей дисциплине:
1. Доля посешенных лекций в первом промежутке ($\frac {посещено} {общее\_количество}$)
2. Доля посешенных лекций в первом промежутке ($\frac {посещено} {общее\_количество}$)
3. Доля посешенных лекций в третьем промежутке ($\frac {посещено} {общее\_количество}$)
4. Доля посешенных практик в первом промежутке ($\frac {посещено} {общее\_количество}$)
5. Доля посешенных практик в первом промежутке ($\frac {посещено} {общее\_количество}$)
6. Доля посешенных практик в третьем промежутке ($\frac {посещено} {общее\_количество}$)
7. Оценка за КТ 1 ($\frac {оценка} {5}$)
8. Оценка за КТ 2 ($\frac {оценка} {5}$)
9. Оценка за экзамен \[2,5\] если зачет \[2;5\] ($\frac {оценка} {5}$)
10. Средний балл по контрольным ($\frac {балл} {5}$)
11. Доля выполненных лабораторных работ ($\frac {выполнено} {общее\_количество}$)
12. Уровень корреляции дисциплины с прогнозируемой



Также идут следующие метки:
- Реальная оценка по дисциплине

На выходе:
- Уровни уверенности в прогнозируемой оценке

In [3]:
model = keras.Sequential([
    layers.Dense(12, input_dim=12, activation='elu'),
    layers.Dense(4, activation='sigmoid')
])

model.compile(optimizer='adam', loss = losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

## Ниже идет баловство со случайными данными.

In [4]:
# Генерация
size = 10000
data = np.random.random((size, 12))
for i in range(size): 
    for j in range(12):
        if (data[i][j] < 2/5) and (j != 11):
            data[i][j] = 2/5

labels = np.random.randint(size=(size, 1), low=0, high=4)

#for i in range(size):
#    labels[i][0] = 1

In [5]:
# Обучение
model.fit(data, labels, epochs=5, batch_size=32)

Epoch 1/5


C:\Users\Qwery\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 1ms/step - loss: 1.4064 - accuracy: 0.2418
Epoch 2/5
313/313 [==============================] - 0s 1ms/step - loss: 1.3965 - accuracy: 0.2425
Epoch 3/5
313/313 [==============================] - 0s 1ms/step - loss: 1.3930 - accuracy: 0.2478
Epoch 4/5
313/313 [==============================] - 0s 1ms/step - loss: 1.3907 - accuracy: 0.2481
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 1.3897 - accuracy: 0.2486


In [6]:
# Оценка качества
size_test = 1000
data_test = np.random.random((size_test, 12))
for i in range(size_test): 
    for j in range(12):
        if (data_test[i][j] < 2/5) and (j != 11):
            data_test[i][j] = 2/5

labels_test = np.random.randint(size=(size_test, 1), low=0, high=4)

loss_and_metrics = model.evaluate(data_test, labels_test, batch_size=32)

32/32 [==============================] - 0s 1ms/step - loss: 1.3909 - accuracy: 0.2440


In [7]:
# Предсказание
prediction = model.predict(data_test)

32/32 [==============================] - 0s 901us/step


# Идея следующего уровня
На вход идет:
1. Средний предсказанный балл
2. Средний уровень корреляции
3. Балл по максимальной корреляции
4. Уровень максимальной корреляции
5. Балл по минимальной корреляции
6. Уровень минимальной корреляции

Метки:
- Реальный полученный балл

На выход:
- Итоговое пердсказание

In [8]:
model1 = keras.Sequential([
    layers.Dense(6, input_dim=6, activation='elu'),
    layers.Dense(4, activation='sigmoid')
])

model1.compile(optimizer='adam', loss = losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])